# Sampling
In this notebook we want to investigate different sampling methods. We start with inverse transform sampling. 
Assume we have a pdf $f(x)$ with a corresponding cdf $F(x) = P(X <= x) = \int_{-\infty}^{x} f(y) dy$. The goal is to draw samples $X\propto f(x)$ from the pdf
distributions in order to compute sample statistics. In this way we can approximate the distribution $f(X)$.

## Motivation
Why do we want to draw samples from distributions:

1. Mathematical expression of f(x) is too complicated (e.g. complext integral definition, etc.) to compute statistics, hence we have to rely on sample statistics.
2. Using sampling techniques we can compute integrals, e.g. with MCMC.
3. In bayesian inference it gives us an approximation of the denominator (which is often intractable)
4. Also important for stochastical gradient descent methods (see https://arxiv.org/abs/1506.03431)

### Sampling from Uniform distribution
This is something we assume that we can do that. It basically deals with random number generators.

### Inverse Transform Sampling

#### Procedure:

1. Take random samples u of a uniform distribution $\mathcal{U}$ between 0 and 1
2. Interprete these samples u as probabilities
3. Return the largest number x from the pdf, such that ${P(-\infty < X < x )} \le u$
4. x is a sample of your pdf

For point 3 you need the inverse of the cdf in order to solve for x.

#### Intuitions
Assume $F(x) = P(X < x)$ is monotonically increasing function (not always true).
We start with a uniform random variate $U \propto \mathcal{U}(0,1)$. Let us define
$X = T(U)$, where $T$ is some function which maps $[0,1] \rightarrow \mathbb{R}$.
In some sense you can interprete $T$ as a function which maps probabilities to the real space.
The random variate then has a non uniform distribution. We want it to follow the distribution $f(x)$, hence
we want $P(T(U)<x) = F(X)$. It turns out that $T(U) = F^{-1}(U)$ does the trick:

$$P(F^{-1}(U)<x) = P(U<F(x)) = F(x)$$, where we used
$P(U < y) = y$ if $U$ is uniform: *The probability of drawing a number smaller than y from a uniform distribution is y*. (nice explanations are here: https://www.quora.com/What-is-an-intuitive-explanation-of-inverse-transform-sampling-method-in-statistics-and-how-does-it-relate-to-cumulative-distribution-function)



In [1]:
import numpy as np
import pandas as pd